<a href="https://colab.research.google.com/github/esequielsantos/ediBot/blob/main/EdiBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar Gemini

---



In [2]:
!pip install -U google-generativeai

## Set up my API key

To use the Gemini API, you'll need an API key. If you don't already have one, create a key in Google AI Studio.

<a class="button" href="https://aistudio.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`. Then pass the key to the SDK:


In [3]:
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Listar modelos disponiveis


In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


Configurando o modelo
Temperaura:

In [5]:
gen_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}

Segurança:

In [6]:
saf_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

Usar **modelo** escolhido acima

In [7]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro-latest', safety_settings= saf_settings, generation_config= gen_config)

Enviando mensagem ao modelo

In [ ]:
response = model.generate_content("Estou programndo em Phyton. Sugira 5 opções indispensáveis de conhecermos no aprendizado de Phyton")
#print(model.generate_content("Estou programndo em Phyton. Sugira 5 opções indispensáveis de conhecermos no aprendizado de Phyton").text)

Exibindo Resposta

In [ ]:
print(response.text)

5 Opções Indispensáveis para o Aprendizado de Python:

1. **Listas de Compreensão:** Uma sintaxe concisa para criar novas listas a partir de elementos existentes, filtrando e transformando-os.
2. **Dicionários:** Estruturas de dados que mapeiam chaves para valores, permitindo acesso rápido e fácil a dados com base em chaves.
3. **Funções Lambda:** Funções anônimas de uma linha que podem ser usadas como argumentos para outras funções ou armazenadas em variáveis.
4. **Geradores:** Objetos iteráveis que produzem valores sob demanda, economizando memória e melhorando a eficiência.
5. **Módulos:** Arquivos Python que contêm funções, classes e variáveis que podem ser importados para reutilização em outros programas.




> Adicionar aspas INICIANDO O EdiBot

**Iniciando**



In [17]:
chat = model.start_chat(history=[])

In [18]:
from datetime import datetime

In [21]:
def edibot(prompt, history):
  if not chat.history:
    # Obter o horário atual
    agora = datetime.now()

    # Extrair a hora do horário atual
    hora = agora.hour

    # Definir a variável do período do dia
    periodo_dia = ""

    # Verificar o período do dia
    if 6 <= hora < 12:
      periodo_dia = "manhã"
    elif 12 <= hora < 18:
      periodo_dia = "tarde"
    else:
      periodo_dia = "noite"

    bemvindo = model.generate_content(f"Você é um atendente de uma empresa de informatica chamada ByteMe chamado EdiBot. Escreva uma mensagem de saudação de {periodo_dia} ").text
    response = chat.send_message(prompt)
    return bemvindo + '\n\n-----------------------\n' + response.text
  else:
    prompt = prompt.lower()
    agradecer = model.generate_content("Você é um atendente de uma empresa de informatica chamada ByteMe. Escreva uma mensagem curta de agradecimento por ter entrado em contato").text
    if prompt == "history":
        return chat.history
    elif 'obrigado' in prompt:
      return agradecer
    elif 'tchau' in prompt:
      return agradecer
    elif 'fim' in prompt:
      return agradecer
    elif prompt == "next":
      continuar = model.generate_content("Escreva uma mensagem curta e agradavél perguntando se deseja mais alguma informação").text
      return continuar
    else:
      response = chat.send_message(prompt)
      return response.text

Teste da funcao inteligente

In [20]:
print(edibot("Bom dia, preciso de um computador novo para jogar", []))

Olá! Seja muito bem-vindo(a) à ByteMe, sou o EdiBot, seu atendente virtual. Como posso ajudá-lo(a) nesta tarde?

-----------------------**Requisitos para um Novo Computador**

**Uso Pretendido:**
* Quais tarefas você pretende realizar no computador? (Trabalho, estudos, jogos, edição de vídeo, etc.)

**Orçamento:**
* Quanto você está disposto a gastar em um novo computador?

**Especificações Recomendadas:**

**Processador (CPU):**
* Intel Core i5 ou i7 (11ª geração ou superior)
* AMD Ryzen 5 ou 7 (5000 Series ou superior)

**Memória RAM:**
* 8 GB (mínimo)
* 16 GB (recomendado)

**Armazenamento:**
* SSD de 512 GB (mínimo)
* SSD de 1 TB (recomendado)

**Placa Gráfica (GPU):**
* Para jogos: NVIDIA GeForce GTX 1650 ou superior
* Para edição de vídeo: NVIDIA GeForce RTX 3050 ou superior

**Sistema Operacional:**
* Windows 11 (recomendado)
* macOS (para usuários Apple)

**Outros Recursos:**
* Tela: 15-17 polegadas com resolução Full HD (1920x1080)
* Teclado e mouse ergonômicos
* Conectividade

In [22]:
print(edibot("Bom dia, preciso de um computador novo para jogar", []))

**Requisitos para um Computador para Jogos**

**Especificações Recomendadas:**

**Processador (CPU):**
* Intel Core i5 ou i7 (12ª geração ou superior)
* AMD Ryzen 5 ou 7 (5000 Series ou superior)

**Memória RAM:**
* 16 GB (mínimo)
* 32 GB (recomendado)

**Armazenamento:**
* SSD de 1 TB (mínimo)
* SSD NVMe de 2 TB (recomendado)

**Placa Gráfica (GPU):**
* NVIDIA GeForce RTX 3060 ou superior
* AMD Radeon RX 6600 XT ou superior

**Sistema Operacional:**
* Windows 11 (recomendado)

**Outros Recursos:**

* Tela: 15-17 polegadas com resolução Full HD (1920x1080) ou superior
* Taxa de atualização: 144 Hz ou superior
* Teclado e mouse para jogos
* Conectividade: Wi-Fi 6, Bluetooth 5.0
* Refrigeração: Sistema de resfriamento eficiente

**Marcas Recomendadas:**

* Alienware
* ASUS ROG
* Corsair
* MSI
* Razer

**Dicas para Escolher um Computador para Jogos:**

* Determine a resolução e a taxa de quadros desejadas para seus jogos.
* Pesquise diferentes placas gráficas e compare seu desempenho.
* C

Subir para o GitHub